In [ ]:
!pip install scikit-surprise
!pip install optuna

     |████████████████████████████████| 11.8MB 4.0MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617603 sha256=fac166ba920b021c51f598562bb471f72b23b706dbc578b78d3fd058709df0db
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise
     |████████████████████████████████| 307kB 3.9MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 174kB 18.7MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 
     |████████████████████████████████| 112kB 20.1MB/s 
     |████████████████████████████████| 143kB 18.6MB/s 
     |████████████████████████████████| 81kB 6.7MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11136 sha256=5670f1cf56b907b3578b611a2f3194388ac0940c19373814e3c5b0600a0b7595
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2

In [ ]:
# Mount Google Drive and set data paths.
import os
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive')
DATA_PATH = "/content/gdrive/My Drive/ETH/Computational Intelligence Lab/CIL-Project/data"
TRAIN_DATA_PATH = os.path.join(DATA_PATH, "data_train.csv")
TEST_DATA_PATH = os.path.join(DATA_PATH, "data_test.csv")

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
import surprise
from surprise import accuracy
from surprise.model_selection import cross_validate
import optuna

In [ ]:
def load_data(file_path: str, full_dataset: bool, train_val_split: bool, random_seed: int = 0, train_size: float = 0):
    data_pd = pd.read_csv(file_path)

    # Reduce Dataset for Testing
    if not full_dataset:
        data_pd = data_pd.head(10000)

    if train_val_split:
        train_pd, val_pd = train_test_split(data_pd, train_size=train_size, random_state=random_seed)
        return train_pd, val_pd
    else:
        return data_pd

def __extract_users_items_ratings(data_pd: pd.DataFrame):
    users, movies = \
        [np.squeeze(arr) for arr in np.split(data_pd.Id.str.extract('r(\d+)_c(\d+)').values.astype(int) - 1, 2, axis=-1)]
    ratings = data_pd.Prediction.values
    return users, movies, ratings

def create_surprise_data(data_pd):
    users, movies, ratings = __extract_users_items_ratings(data_pd)

    df = pd.DataFrame({
        'users': users,
        'movies': movies,
        'ratings': ratings
    })
    reader = surprise.Reader(rating_scale=(1, 5))
    return surprise.Dataset.load_from_df(df[['users', 'movies', 'ratings']], reader=reader)

def __get_tensors_from_dataframe(data_pd: pd.DataFrame):
    users, movies, ratings = __extract_users_items_ratings(data_pd)
    users_torch = torch.tensor(users, dtype=torch.int64)
    movies_torch = torch.tensor(movies, dtype=torch.int64)
    ratings_torch = torch.tensor(ratings, dtype=torch.int64)

    return users_torch, movies_torch, ratings_torch


def create_dataset(data_pd: pd.DataFrame, test_dataset: bool = False):
    users_torch, movies_torch, ratings_torch = __get_tensors_from_dataframe(data_pd)

    if not test_dataset:
        return TensorDataset(users_torch, movies_torch, ratings_torch)
    else:
        test_ids = data_pd.Id
        return test_ids, TensorDataset(users_torch, movies_torch)

In [ ]:
random_seed = 42
full_dataset = True

np.random.seed(random_seed)

train_pd = load_data(
    file_path=TRAIN_DATA_PATH,
    full_dataset=full_dataset,
    train_val_split=False
)

In [ ]:
train_data = create_surprise_data(train_pd)
trainset, testset = surprise.model_selection.train_test_split(train_data, test_size=0.2)

In [ ]:
from surprise import SVD

def objective(trial):
    n_factors = trial.suggest_int("n_factors", 2, 500) #default 100
    n_epochs = trial.suggest_int("n_epochs", 10, 30) #default 20
    init_mean = trial.suggest_int("init_mean", 0, 5) #default 0
    init_std_dev = trial.suggest_float("init_std_dev", 0, 1) #default 0.1
    lr_all = trial.suggest_float("lr_all", 0, 1e-2) #default 0.005
    reg_all = trial.suggest_float("reg_all", 0, 1e-1) #default 0.02

    algo = SVD(biased=False, n_factors=n_factors, n_epochs=n_epochs, init_mean=init_mean, init_std_dev=init_std_dev, lr_all=lr_all, reg_all=reg_all)
    
    algo.fit(trainset)
    predictions = algo.test(testset)
    rmse = accuracy.rmse(predictions)
    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")

[I 2021-06-08 19:27:34,580] A new study created in memory with name: no-name-6d72f57b-01f3-44ee-9e68-78d5cd27a39e


In [ ]:
# Enqueue trials with default parameters.
study.enqueue_trial(
    {
        "n_factors": 100,
        "n_epochs": 20,
        "init_mean": 0,
        "init_std_dev": 0.1,
        "lr_all": 0.005,
        "reg_all": 0.02
    }
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: ExperimentalWarning:

enqueue_trial is experimental (supported from v1.2.0). The interface can change in the future.

/usr/local/lib/python3.7/dist-packages/optuna/study.py:858: ExperimentalWarning:

create_trial is experimental (supported from v2.0.0). The interface can change in the future.

/usr/local/lib/python3.7/dist-packages/optuna/study.py:858: ExperimentalWarning:

add_trial is experimental (supported from v2.0.0). The interface can change in the future.



In [ ]:
study.optimize(objective, n_trials=100)

[I 2021-06-08 19:28:34,514] Trial 0 finished with value: 1.0327349604267961 and parameters: {'n_factors': 100, 'n_epochs': 20, 'init_mean': 0, 'init_std_dev': 0.1, 'lr_all': 0.005, 'reg_all': 0.02}. Best is trial 0 with value: 1.0327349604267961.


RMSE: 1.0327


[I 2021-06-08 19:30:11,828] Trial 1 finished with value: 1.1726615088368082 and parameters: {'n_factors': 292, 'n_epochs': 13, 'init_mean': 0, 'init_std_dev': 0.22602245437744117, 'lr_all': 0.003304524580820677, 'reg_all': 0.003220847700665708}. Best is trial 0 with value: 1.0327349604267961.


RMSE: 1.1727


[I 2021-06-08 19:31:00,087] Trial 2 finished with value: 1.5971405493085367 and parameters: {'n_factors': 150, 'n_epochs': 12, 'init_mean': 4, 'init_std_dev': 0.9729636789560008, 'lr_all': 0.0011966039763301117, 'reg_all': 0.04133579844443397}. Best is trial 0 with value: 1.0327349604267961.


...

RMSE: 1.2950


[I 2021-06-08 20:50:22,499] Trial 98 finished with value: 1.0068513496189553 and parameters: {'n_factors': 29, 'n_epochs': 16, 'init_mean': 0, 'init_std_dev': 0.15867181186752954, 'lr_all': 0.0012924029197044107, 'reg_all': 0.013754723284689864}. Best is trial 95 with value: 1.0038025058306475.


RMSE: 1.0069


[I 2021-06-08 20:53:44,940] Trial 99 finished with value: 1.0260055320411121 and parameters: {'n_factors': 403, 'n_epochs': 18, 'init_mean': 0, 'init_std_dev': 0.10646622297116701, 'lr_all': 0.00207936833314455, 'reg_all': 0.015159933774924939}. Best is trial 95 with value: 1.0038025058306475.


RMSE: 1.0260


In [ ]:
# NOTE: We do not prune any trials since surprise does not support partial fit.
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  100
  Number of pruned trials:  0
  Number of complete trials:  100
Best trial:
  Value:  1.0038025058306475
  Params: 
    n_factors: 30
    n_epochs: 14
    init_mean: 0
    init_std_dev: 0.052150520280646796
    lr_all: 0.00202934716100709
    reg_all: 0.014927795931791398


In [ ]:
test_pd = load_data(
    file_path=TEST_DATA_PATH,
    full_dataset=full_dataset,
    train_val_split=False
)

train_data = create_surprise_data(train_pd).build_full_trainset()
test_ids, test_data = create_dataset(test_pd, test_dataset=True)
test_ids = test_ids.to_numpy()

from surprise import SVD

n_factors = 30
n_epochs = 14
init_mean = 0
init_std_dev = 0.052150520280646796
lr_all = 0.00202934716100709
reg_all = 0.014927795931791398

algo = SVD(biased=False, n_factors=n_factors, n_epochs=n_epochs, init_mean=init_mean, init_std_dev=init_std_dev, lr_all=lr_all, reg_all=reg_all)

algo.fit(train_data)

predictions = []
for user, movie in test_data:
    prediction = algo.predict(user.item(), movie.item()).est
    predictions.append(prediction)

output = np.stack((test_ids, predictions), axis=1)

pd.DataFrame(output, columns=["Id", "Prediction"]).to_csv("svd_unbiased_output.csv", index=None)
files.download("svd_unbiased_output.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>